In [217]:
import pandas as pd
path = r'C:\Users\user\Desktop\HELP\count16.xlsx'
df = pd.read_excel(path)
df = df[df["g"].notna()]


In [218]:
df["g"]

2                      資料釋出的時效性，許多資料雖為近期釋出，但實際執行年度距今已久遠
6     國科會能多方採納及鼓勵及補助具有意義的長期追蹤及家戶為單位的資料庫,例如超高齡社會之緊急社會...
24    與全球資料庫發展長期穩定合作關係，例如世界價值觀調查曾經在台灣無人接手，導致資料永遠缺失無法彌補。
26    增加多一點的新進樣本。不然多年後樣本可以年齡層會不符合當下的世代。例如台灣青少年發展歷程中的...
32                                     Provide NHI data
40                                                    無
63                                        需要更多社會政策相關的題項
64    1.實際看社會需要發展做長期規劃 2.新住民研究是一種標籤，名稱應該為弱勢族群比較好 ，而且...
68    建議相關資料宣傳推廣到大專院校研究生，讓更多研究生知道政府提供哪些完整的資料。甚至可以大數據...
79                                         希望會員可以不用定期更新
83                      非常非常希望快速一點公告資料集，太老舊的資料分析起來常覺得汗顏
90                                          長期性的研究能持續進行
92                新住民是個標籤應採納在不同領域下，不太適宜的主題容易污名化恐失去原有的善意
93    國民健康署的資料庫也相當豐富，但一直在如何申請上有一些問題。SRDA在資料庫存取都相當成熟，...
94                                  請宣導推廣資料庫，以增加資料庫使用率。
Name: g, dtype: object

In [219]:
import jieba
stop_path = r'C:\Users\user\Desktop\py\wordcloud\stop1.txt'

def stopwordslist(stop_path):
    stopwords = [line.strip() for line in open(stop_path, 'r',
                                               encoding='utf-8-sig', errors='ignore').readlines()]
    return stopwords

def seg_sentence(sentence):
    sentence_seged = jieba.cut(sentence.strip())
    stopwords = stopwordslist(stop_path)  # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr


df['gg'] = df["g"].astype(str).apply(seg_sentence)
df['gg']

2                  釋出 時 效性 許多 雖為 近期 釋出 實際 執行 年度 距今 已久遠 
6     國科會 能 多方 採納及 補助 具有 意義的 長 期 追 蹤 家 戶 單 位 資 料 庫 超...
24    全球 庫 發展 長 期 穩定 合作 關 係 世界 價值觀 曾經 無人 接手 導致 缺失 無法彌補 
26    增加 多一點 新 進樣 本 不然 多年 樣本 年齡 層會 符合 當下 世代 青少年 發展 歷...
32                                     Provide   NHI   
40                                                     
63                                         更多 社會 政策 題項 
64    1 實際 社會 發展 做長 期規劃   2 新 住民 一種 標 籤 名稱 應該 弱勢 族群 ...
68    宣傳 推廣到 大專 院校 研究生 更 多 研究生 知道 政府 提供 完整 甚至 大數據 統計...
79                                         會員 不用 定期 更新 
83                         快速 一點 公告 集 太老舊 分析 起來 常覺 汗 顏 
90                                            長 能持續 進行 
92        新 住民 個標 籤 應 採納 不同 不太 適宜 主題 容易 污名 化恐 失去 原有 善意 
93    國民 健康 署 庫 豐富 一直 如何 申請 一些 問題 庫 存取 成熟 其實 政府 單位 應...
94                                請 宣導 推廣資料 庫 增加 庫 使用率 
Name: gg, dtype: object

In [220]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

n_features = 50

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.9,
                                min_df = 1)
tf = tf_vectorizer.fit_transform(df['gg'])

In [221]:
from sklearn.decomposition import LatentDirichletAllocation

n_components=5
lda = LatentDirichletAllocation(n_components=n_components, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)

def print_top_words(model, feature_names, n_components):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_components - 1:-1]]))
    print()

In [222]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_components)

Topic #0:
青少年 社會 分析 年齡 樣本
Topic #1:
能持續 政府 問題 進行 放在
Topic #2:
晚生 年齡 弱勢 許多 接手
Topic #3:
實際 弱勢 住民 釋出 族群
Topic #4:
研究生 統計 院校 進行 更新



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [223]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.129094  0.009728       1        1  57.154232
3      0.041311 -0.114552       2        1  19.828769
4      0.070761  0.071162       3        1  13.167686
1      0.025406  0.036546       4        1   7.528128
2     -0.008385 -0.002884       5        1   2.321185, topic_info=   Term      Freq     Total Category  logprob  loglift
36  研究生  1.000000  1.000000  Default  30.0000  30.0000
5    問題  1.000000  1.000000  Default  29.0000  29.0000
41   進行  1.000000  1.000000  Default  28.0000  28.0000
38   統計  1.000000  1.000000  Default  27.0000  27.0000
19   政府  1.000000  1.000000  Default  26.0000  26.0000
..  ...       ...       ...      ...      ...      ...
5    問題  0.040362  1.694176   Topic5  -3.9127   0.0260
33   現象  0.040780  2.112364   Topic5  -3.9024  -0.1843
7    太少  0.040597  2.115403   Topic5  -3.9069  -0.1902
37   社會  0.041170  4.147814   Topic5  -3.8929  -0.8495
49  青少年  0.040211  4.326541   Topic5  -3.9164  -0.9153

[219 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         1  0.947356    主題
1         1  0.403370    住民
1         2  0.403370    住民
2         1  0.545889   使用率
2         3  0.545889   使用率
3         1  0.840579    分析
4         1  0.945376    動態
5         1  0.590257    問題
6         1  0.947269    增加
7         1  0.945446    太少
8         1  0.949142    家庭
9         2  0.572993    實際
10        1  0.698113    年齡
11        2  0.573150    弱勢
12        1  0.741666    採納
13        1  0.725787   採納及
14        1  0.720612    接手
15        3  0.931097   推廣到
16        1  0.716704  推廣資料
17        3  0.927685    提供
19        3  0.714432    政府
20        1  0.726623    政策
21        2  0.839787    效性
22        2  0.577962    族群
23        1  0.735636    晚婚
24        1  0.735124    晚生
25        1  0.736893    更多
26        3  0.925852    更新
27        1  0.730499    曾經
28        3  0.930494    會員
29        2  0.842394   期規劃
30        1  0.704740    樣本
31        1  0.726387    步入
32        1  0.731142    歷程
33        1  0.946807    現象
34        1  0.754574    發展
34        2  0.377287    發展
35        1  0.953459    目前
36        3  0.655609   研究生
37        1  0.723273    社會
37        2  0.241091    社會
38        3  0.709410    統計
40        1  0.518127    許多
40        2  0.518127    許多
41        3  0.714247    進行
42        1  0.736460    適宜
43        2  0.577453    釋出
44        2  0.846824    重視
45        1  0.726649    限制
46        3  0.924833    院校
47        2  0.854375    雖為
48        1  0.726703    需求
49        1  0.924526   青少年, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 5, 2, 3])